# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [124]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [22]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [23]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [24]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh0NSApAMFIAAGyp0poAAAAX","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":931112759,"wgRevisionId":931112759,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Article

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [25]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dm


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [26]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [27]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:5]:
    print(pTag.text)

Content analysis is a research method for studying documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in conten

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [28]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is a research method for stud...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   More generally, content analysis is research u...
7   By having contents of communication available ...
8   Robert Weber notes: "To make valid inferences ...
9   There are five types of texts in content analy...
10  Over the years, content analysis has been appl...
11  In recent times, particularly with the advent ...
12  Quantitative content analysis has enjoyed a re...
13  Content analysis can also be described as stud...
14  Holsti groups fifteen uses of content analysis...
15  He also places these uses into the context of ...
16  The following table shows fifteen uses of cont...
17  The process of the initi

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [29]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [30]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [31]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

## Solution
My anticipated final project consists of analyzing the content of the Annual Presidential Speech
of Peru (a speech analogous to the US State of the Union) from 1980 to 2019, given annually by the president of Peru every July 28 (the celebration of Peru's independence claim from Spain).

As the text of every speech is published online in the website of the Congress of Peru in separate pdf documents, I will not use them right now for the purpose of this task. Instead, I will download and parse the Wikipedia article on the President of Peru in Spanish, which is a web content closely related to my anticipated final project.

First, I define some text variables I will use:

In [32]:
wikiPresidPeru_url = 'https://es.wikipedia.org/wiki/Presidente_del_Per%C3%BA'
wikiPresidPeru_save = 'wikiPresidPeru.html'

As you can see, this leads as to a pdf file rather than to an html page. I'll use the library `pdfminer` to parse its content.

In [33]:
wikiPresidPeruRequest = requests.get(wikiPresidPeru_url)
wikiPresidPeruRequest

<Response [200]>

The request was successful. Now I'll parse the content using Beautiful Soup.

In [34]:
wikiPresidPeruSoup = bs4.BeautifulSoup(wikiPresidPeruRequest.text, 'html.parser')
print(wikiPresidPeruSoup.text[:200])





Presidente del Perú - Wikipedia, la enciclopedia libre
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable"


Looks good. Now we save this result in an html file:

In [35]:
with open(wikiPresidPeru_save, mode='w', encoding='utf-8') as f:
    f.write(wikiPresidPeruRequest.text)

Then, I'll save the paragraph tags in a separate object and visualize the first two.

In [36]:
wikiPresidPeruPTags = wikiPresidPeruSoup.body.findAll('p')
for pTag in wikiPresidPeruPTags[:2]:
    print(pTag.text)

El presidente de la República del Perú es el jefe de Estado y de Gobierno del país. Personifica a la nación peruana, es el jefe del poder ejecutivo y jefe supremo de las Fuerzas Armadas y Policiales del Perú. Su cargo corresponde a la más alta magistratura del país y al funcionario público de mayor jerarquía.

La presidencia del Perú es ejercida por un período de 5 años sin posibilidad de reelección inmediata. Luego de un período constitucional, como mínimo, quien haya ejercido el cargo puede volver a postular. El cambio de mando se realiza cada cinco años, el día 28 de julio, en el Congreso de la República. El actual presidente constitucional de la República desde el 23 de marzo de 2018 es Martín Vizcarra Cornejo, quien siguiendo la línea de sucesión presidencial asumió el mando luego de que fuera aceptada la renuncia del hasta entonces presidente Pedro Pablo Kuczynski,[1]​ debiendo completar el periodo presidencial iniciado el 28 de julio de 2016 hasta el 28 de julio de 2021.



After this, we can save the paragraphs in a Pandas dataframe. I'll use regular expressions to delete the references and to replace the graphical accents used in Spanish with the vowels a, e, i, o and u.

In [37]:
wikiPresidPeruParagraphs = []
for pTag in wikiPresidPeruPTags:
    paragraph = pTag.text
    paragraph = re.sub(r'\[\d+\]', '', paragraph)
    paragraph = re.sub('á', 'a', paragraph)
    paragraph = re.sub('é', 'e', paragraph)
    paragraph = re.sub('í', 'i', paragraph)
    paragraph = re.sub('ó', 'o', paragraph)
    paragraph = re.sub('ú', 'u', paragraph)
    paragraph = re.sub('Á', 'A', paragraph)
    paragraph = re.sub('É', 'E', paragraph)
    paragraph = re.sub('Í', 'I', paragraph)
    paragraph = re.sub('Ó', 'O', paragraph)
    paragraph = re.sub('Ú', 'U', paragraph)
    wikiPresidPeruParagraphs.append(paragraph)

#convert to a DataFrame
wikiPresidPeruParagraphsDF = pandas.DataFrame({'paragraph-text' : wikiPresidPeruParagraphs})
wikiPresidPeruParagraphsDF[:10]

,paragraph-text
0,El presidente de la Republica del Peru es el j...
1,La presidencia del Peru es ejercida por un per...
2,"Le corresponde, como mandato generico, dirigir..."
3,El presidente ejerce sus funciones desde la Ca...
4,Recibe el tratamiento protocolar de Excelentis...
5,El primer Estado reconocible como tal bajo con...
6,El Estado peruano moderno es heredero del Virr...
7,"En 1542, se establecio el Virreinato del Peru ..."
8,Este periodo tuvo solo dos etapas correspondie...
9,El virrey Jose Fernando de Abascal fue el enca...


This is nice. Just for fun, I'll repeat this operation to extract all the section headers in this page.

In [38]:
wikiPresidPeru_h2Tags = wikiPresidPeruSoup.body.findAll('h2')
wikiPresidPeruHeaders = []
headers_to_skip = ['Indice', 'Vease tambien', 'Referencias', 'Enlaces externos', 'Menu de navegacion']
# list of headers that appear in every wikipedia page

for h2Tag in wikiPresidPeru_h2Tags:
    header = h2Tag.text
    header = re.sub(r'\[\d+\]', '', header)
    header = re.sub(r'\[editar\]', '', header)
    header = re.sub('á', 'a', header)
    header = re.sub('é', 'e', header)
    header = re.sub('í', 'i', header)
    header = re.sub('ó', 'o', header)
    header = re.sub('ú', 'u', header)
    header = re.sub('Á', 'A', header)
    header = re.sub('É', 'E', header)
    header = re.sub('Í', 'I', header)
    header = re.sub('Ó', 'O', header)
    header = re.sub('Ú', 'U', header)
    if not header in headers_to_skip:
        wikiPresidPeruHeaders.append(header)

#convert to a DataFrame
wikiPresidPeruHeadersDF = pandas.DataFrame({'header-text' : wikiPresidPeruHeaders})
wikiPresidPeruHeadersDF

,header-text
0,Antecedentes
1,Funciones
2,Requisitos
3,Eleccion
4,Duracion del mandato y posibilidad de reeleccion
5,Reemplazo temporal o permanente
6,Ceremonias protocolares
7,Simbolos
8,Expresidentes
9,Primera Dama



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [39]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [40]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,6
7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,8
9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [41]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [42]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



## Solution

For this exercise, I will follow the URL stored in the object `wikiPresidPeru_url`. Then, I will create a recursive function to do the following:

* The function will check the current "deepness level" of the URL we are accessing, with respect of the link that was provided at the beginning of the recursion. If that level is equal to zero, it will return and empty dataframe.
* If the level is not zero, the function will check if the URL is already on the list of previously accessed URLs. If it is, it will return an empty dataframe.
* If the URL has not been accessed before, the function will (i) add the paragraphs of this page in a dataframe, (ii) add the URL to the list of URLs visited, (iii) get a list of the additional URLs found in those paragraphs, and (iv) recursively access those additional URLs to access their paragraphs to the dataframe created in (i). Finally, the function will return this dataframe.

To begin with, I will define some helper functions I will use in my recursive Spidering function.

1. Function to get the paragraph tags in the text of a URL:

In [45]:
def get_PTags(url):
    
    request = requests.get(url)
    soup = bs4.BeautifulSoup(request.text, 'html.parser')
    PTags = soup.body.findAll('p')

    return PTags

2. Function to get a list of the URLs found in a list of paragraph tags:

In [57]:
def get_otherPageURLs(PTags):
    
    otherPageURLs = []

    for paragraphNum, pTag in enumerate(PTags):

        #we only want hrefs that link to wiki pages
        tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    
        for aTag in tagLinks:

            relurl = aTag.get('href')
            otherPageURLs.append(urllib.parse.urljoin('https://es.wikipedia.org', relurl))
        
    return otherPageURLs

3. Function to extract the text of a list of paragraph tags:

In [46]:
def get_paragraphs(PTags):

    paragraphs = []

    for pTag in PTags:
        
        paragraph = pTag.text
        paragraph = re.sub(r'\[\d+\]', '', paragraph)
        paragraph = re.sub('á', 'a', paragraph)
        paragraph = re.sub('é', 'e', paragraph)
        paragraph = re.sub('í', 'i', paragraph)
        paragraph = re.sub('ó', 'o', paragraph)
        paragraph = re.sub('ú', 'u', paragraph)
        paragraph = re.sub('Á', 'A', paragraph)
        paragraph = re.sub('É', 'E', paragraph)
        paragraph = re.sub('Í', 'I', paragraph)
        paragraph = re.sub('Ó', 'O', paragraph)
        paragraph = re.sub('Ú', 'U', paragraph)
        paragraphs.append(paragraph)

    return paragraphs

Having defined these function, now I'll build the recursive Spidering function, starting from a given URL. This function will "go deeper" in the links a certain number of times, defined in the argument `current_level`. Importantly, the function will also check that every URL is not a URL previously accessed. Finally, it will return a dataframe of paragraphs with:

1. The source URL of the paragraph
2. The number of the paragraph within the page it belongs to
3. The text of the paragraph itself

In [60]:
def spider_OtherPagesParagraphs(url, current_level):
    
    parsDict = {'source' : [],
                'paragraph-number' : [],
                'paragraph-text' : []}
    df = pandas.DataFrame(parsDict)

    if current_level == 0:
        
        return df

    else:
        
        if not url in visited_urls:

            PTags = get_PTags(url)
            URLs = get_otherPageURLs(PTags)
            paragraphs = get_paragraphs(PTags)

            for i, paragraph in enumerate(paragraphs):
                    
                df = df.append({'source': url,
                                'paragraph-number': i + 1,
                                'paragraph-text': paragraph},
                                ignore_index=True)
                
            visited_urls.add(url)

            for URL in URLs:
            
                df = df.append(spider_OtherPagesParagraphs(URL, current_level - 1),
                               ignore_index=True)

        return df

Finally, I'll try this function with our `wikiPresidPeru_url` object. I'll set the `current_level` value to 2, meaning that it will only go one level deep in the recursion.

In [61]:
visited_urls = set()
spider_OtherPagesParagraphs(wikiPresidPeru_url, 2)

,source,paragraph-number,paragraph-text
0,https://es.wikipedia.org/wiki/Presidente_del_P...,1.0,El presidente de la Republica del Peru es el j...
1,https://es.wikipedia.org/wiki/Presidente_del_P...,2.0,La presidencia del Peru es ejercida por un per...
2,https://es.wikipedia.org/wiki/Presidente_del_P...,3.0,"Le corresponde, como mandato generico, dirigir..."
3,https://es.wikipedia.org/wiki/Presidente_del_P...,4.0,El presidente ejerce sus funciones desde la Ca...
4,https://es.wikipedia.org/wiki/Presidente_del_P...,5.0,Recibe el tratamiento protocolar de Excelentis...
5,https://es.wikipedia.org/wiki/Presidente_del_P...,6.0,El primer Estado reconocible como tal bajo con...
6,https://es.wikipedia.org/wiki/Presidente_del_P...,7.0,El Estado peruano moderno es heredero del Virr...
7,https://es.wikipedia.org/wiki/Presidente_del_P...,8.0,"En 1542, se establecio el Virreinato del Peru ..."
8,https://es.wikipedia.org/wiki/Presidente_del_P...,9.0,Este periodo tuvo solo dos etapas correspondie...
9,https://es.wikipedia.org/wiki/Presidente_del_P...,10.0,El virrey Jose Fernando de Abascal fue el enca...


It works! We see that going only one level deep, it stored the content of 5800+ paragraphs.

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [62]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3463,"posts-type":false,"posts":[{"id":"190233110922","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190233110922","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190233110922\/a-most-fortunate-red-headed-stepkitten","type":"photo","date-gmt":"2020-01-13 11:03:57 GMT","date":"Mon, 13 Jan 2020 12:03:57","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1578913437,"format":"html","reblog-key":"BRsWJpjQ","slug":"a-most-fortunate-red-headed-stepkitten","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"190233110922\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_190233110922\"><iframe id=\"like_iframe_190233110922\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [63]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [64]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [65]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,190233110922,https://66.media.tumblr.com/42458742dedc510f3d...,"Mon, 13 Jan 2020 12:03:57","[cat, cats, lol, lolcat, lolcats]",jpg
1,190232075547,https://66.media.tumblr.com/65b1e6efd6ec145872...,"Mon, 13 Jan 2020 10:00:35","[cat, cats, lol, lolcat, lolcats]",jpg
2,190230902981,https://66.media.tumblr.com/616b4faab5a289a369...,"Mon, 13 Jan 2020 08:01:02","[cat, cats, lol, lolcat, lolcats]",jpg
3,190229342226,https://66.media.tumblr.com/28121ad01b05305b89...,"Mon, 13 Jan 2020 06:00:57","[cat, cats, lol, lolcat, lolcats]",jpg
4,190209640041,https://66.media.tumblr.com/9b735740f517f3673b...,"Sun, 12 Jan 2020 06:01:05","[cat, cats, lol, lolcat, lolcats]",jpg
5,190193496849,https://66.media.tumblr.com/1ffa3c880a55f80f62...,"Sat, 11 Jan 2020 10:01:29","[cat, cats, lol, lolcat, lolcats]",jpg
6,190192270965,https://66.media.tumblr.com/a5ecfce3b06defdbac...,"Sat, 11 Jan 2020 08:00:47","[cat, cats, lol, lolcat, lolcats]",jpg
7,190190785530,https://66.media.tumblr.com/522386ce0644901d60...,"Sat, 11 Jan 2020 06:01:02","[cat, cats, lol, lolcat, lolcats]",jpg
8,190174143051,https://66.media.tumblr.com/5f1a4eec3910ec2262...,"Fri, 10 Jan 2020 08:00:54","[cat, cats, lol, lolcat, lolcats]",jpg
9,190153974947,https://66.media.tumblr.com/e70ac81ae13bf839f8...,"Thu, 09 Jan 2020 06:01:10","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [66]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [67]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [68]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

FileNotFoundError: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt'

Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [69]:
information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@R�Q`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [73]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams) #, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [71]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [75]:
print(readPDF(infoExtractionBytes)[:560])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determining the rel


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [76]:
example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [77]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [78]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [79]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


## Solution

As I mentioned in Section 1, my anticipated project consists on analyzing the Annual Presidential Speeches of the President of Peru. They are published in the website of the Congress of Peru in separate `pdf` files.

For the purpose of this exercise, I'll use the `pdf` of the latest speech, given in July 28th 2019. I'll first define the URL of the file and the name I want to give it.

In [82]:
pdf_url = 'http://www.congreso.gob.pe/Docs/participacion/museo/congreso/files/mensajes/2001-2020/files/mensaje-2019-01-vizcarra.pdf'
pdf_save = 'mensaje-2019-01-vizcarra.pdf'

Now I'll store the first reading of the pdf file in an object named `pdf`.

In [85]:
pdf = downloadIfNeeded(pdf_url, pdf_save, mode = 'rb')

Then, I'll read it directly using the function `readPDF()`.

In [89]:
pdf_text= readPDF(pdf)

In [139]:
pdf_text[:1000]

'DISCURSO DEL PRESIDENTE CONSTITUCIONAL DE LA REPÚBLICA, \n\nINGENIERO MARTÍN ALBERTO VIZCARRA CORNEJO, \n\nANTE EL CONGRESO NACIONAL, EL 28 DE JULIO DE 20191 \n\n \n \nSeñor Presidente del Congreso de la República; \nSeñoras y señores congresistas; \nSeñoras y señores ministros de Estado; \nDistinguidas autoridades civiles y militares; \nMiembros del cuerpo diplomático acreditado en el Perú; \nCompatriotas, a lo largo y ancho del país: \n \nDe conformidad al artículo 118 de la Constitución Política del Estado, me dirijo al \npaís al cumplir un año y cuatro meses al frente del gobierno, para hacer un balance \ndel  camino  recorrido  hasta  hoy  y  compartir  con  todos  los  peruanos  las  próximas \nacciones en el marco de la ruta trazada y expresada en los cinco ejes de nuestra \npolítica de gobierno.  \n \nAl pronunciar este mensaje desde la sede del Congreso, no puedo dejar de recordar \nante  todos  los  peruanos  las  circunstancias  en  que  asumimos  la  conducción  del \ngobi

After watching this small first portion of the text and the pdf file, I can see that every paragraph is separated by a blankspace followed by a new line character ('\n'), another blankspace and another new line character. I'll use this separation to split each paragraph in different elements

In [110]:
paragraphs = re.split(' \n \n', pdf_text)

In [141]:
paragraphs[:5]

['DISCURSO DEL PRESIDENTE CONSTITUCIONAL DE LA REPÚBLICA, \n\nINGENIERO MARTÍN ALBERTO VIZCARRA CORNEJO, \n\nANTE EL CONGRESO NACIONAL, EL 28 DE JULIO DE 20191 \n\n',
 'Señor Presidente del Congreso de la República; \nSeñoras y señores congresistas; \nSeñoras y señores ministros de Estado; \nDistinguidas autoridades civiles y militares; \nMiembros del cuerpo diplomático acreditado en el Perú; \nCompatriotas, a lo largo y ancho del país:',
 'De conformidad al artículo 118 de la Constitución Política del Estado, me dirijo al \npaís al cumplir un año y cuatro meses al frente del gobierno, para hacer un balance \ndel  camino  recorrido  hasta  hoy  y  compartir  con  todos  los  peruanos  las  próximas \nacciones en el marco de la ruta trazada y expresada en los cinco ejes de nuestra \npolítica de gobierno. ',
 'Al pronunciar este mensaje desde la sede del Congreso, no puedo dejar de recordar \nante  todos  los  peruanos  las  circunstancias  en  que  asumimos  la  conducción  del \ngobier

Still, we can see that some paragraphs have trailinig blankspaces and line breaks. Then, we proceed to filter those out.

In [122]:
new_paragraphs = []
for paragraph in paragraphs:
    new_paragraph = re.sub('\n', '', paragraph)
    new_paragraph = re.sub('( )+$', '', new_paragraph)
    new_paragraphs.append(new_paragraph)

Finally, I'll create a dataframe where every element is a paragraph. I'll only include two columns:

* Paragraph number
* Paragraph text

In [133]:
pdf_df = pandas.DataFrame(columns = ['paragraph_num', 'paragraph_text'])
pdf_df['paragraph_text'] = new_paragraphs
pdf_df['paragraph_num'] = range(len(pdf_df))

In [137]:
pdf_df[:10]

,paragraph_num,paragraph_text
0,0,DISCURSO DEL PRESIDENTE CONSTITUCIONAL DE LA R...
1,1,Señor Presidente del Congreso de la República;...
2,2,De conformidad al artículo 118 de la Constituc...
3,3,Al pronunciar este mensaje desde la sede del C...
4,4,"Conscientes del enorme desafío, en estos dieci..."
5,5,Es la democracia y la fe de los peruan...
6,6,"Por eso, no solo hemos garantizado una transic..."
7,7,El objetivo común es generar instituciones sól...
8,8,"Ahora, haré referencia a las variables e..."
9,9,El mayor desafío que nos puso la histor...


Works nicely.

### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

